In [638]:
import pandas as pd

In [639]:
model_responses = pd.read_pickle("model_interpretations.pl")

In [671]:
def get_true_false(annotation):
    try:
        if "seguridad" in annotation.lower():
            return 0
        if "s" in annotation.lower():
            return 1
    except:
        None
    return 0

In [672]:
personajes = pd.read_csv('relacion_personajes.csv')

In [673]:
prod_origen = {"archer": "es", "south park": "la"}

In [674]:
personajes["origen"]= personajes["serie"].map(prod_origen)

In [675]:
sarcasm = pd.read_csv('sarcasmo.csv')

In [676]:
sarcasm = sarcasm.merge(personajes)

In [677]:
sarcasm["respuestas_modelo"] = model_responses.apply(lambda x: x['Sarcastic']>0.5).astype(int)

In [678]:
sarcasm["respuestas_experto"] = sarcasm["Sarcasmo"].astype(int)

In [679]:
sarcasm = sarcasm.fillna("")

In [680]:
def clean_dataset(dataset, name):
    dataset["respuestas_"+name]= dataset["¿Es sarcasmo? (Si, No, No puedo decir con seguridad)"].apply(get_true_false)
    dataset.pop("¿Es sarcasmo? (Si, No, No puedo decir con seguridad)")
    del dataset["Locutor"]
    del dataset["text"]
    try:
        dataset.pop('Unnamed: 3')
    except:
        None
    return dataset

In [681]:
venezuela = clean_dataset(pd.read_excel('anotacion/venezuela.xlsx'), "venezuela")


In [682]:
mexico = clean_dataset(pd.read_excel('anotacion/mexico.xlsx'), "mexico")

In [683]:
argentina_1 = clean_dataset(pd.read_excel('anotacion/argentina/argentina_katie.xlsx'), "argentina_1")

In [684]:
argentina_2 = clean_dataset(pd.read_excel('anotacion/argentina/argentina_jose.xlsx'), "argentina_2")

In [685]:
argentina_3 = clean_dataset(pd.read_excel('anotacion/argentina/argentina_beca.xlsx'), "argentina_3")

In [686]:
argentina_df = pd.concat([argentina_1, argentina_2, argentina_3], axis=1)

In [687]:
sarcasm_base = pd.read_csv('sarcasmo.csv')
sarcasm_base = sarcasm_base.merge(personajes)
sarcasm_base["respuestas_experto"] = sarcasm["Sarcasmo"].astype(int)
sarcasm_base["respuestas_modelo"] = model_responses.apply(lambda x: x['Sarcastic']>0.5).astype(int)
argentina_set = pd.concat([sarcasm_base, argentina_df], axis=1)

In [688]:
argentina_set.to_pickle("argentina_set.pl")

In [689]:
argentina_list = []
for i,x in argentina_df.iterrows():
    argentina_list.append(x.value_counts().keys()[0])
argentina = pd.Series(argentina_list, name="respuestas_argentina")

In [690]:
sarcasm_responses =pd.concat([sarcasm, venezuela, mexico, argentina], axis=1)

In [691]:
len(sarcasm_responses)

959

In [692]:
sarcasm_responses = sarcasm_responses[sarcasm_responses["serie"] != "na"].reset_index(drop=True)

In [693]:
len(sarcasm_responses)

956

In [694]:
sarcasm_responses.to_pickle("prepared_dataset.pl")